In [ ]:
#Split location-only results by MA and cone angle.

In [1]:
import cdflib
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import datetime as dt
import glob
from math import pi
import math
from XMA_finder import XMA_finder
from histo_plot import histo_plot

In [3]:
#open OMNI *average* CSVs
list_all = []

path = "/Users/apx059/Documents/Location_Only_Checks/OMNI Averages/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
om_csvs = []

for element in list_all:
    if '.csv' in element:
        om_csvs.append(element)
        
om_dfs = []

for element in om_csvs:
    om = pd.read_csv(element)
    om_dfs.append(om)
    
omni_all = pd.concat(om_dfs)
omni_all['datetime'] = pd.to_datetime(omni_all['datetime'])

omni_all = omni_all.set_index('datetime')

for element in om_dfs:
    element['datetime'] = pd.to_datetime(element['datetime'])
    element = element.set_index('datetime', inplace = True)


In [4]:
##load Cluster CSVs

list_all = []

path = "/Users/apx059/Documents/23YrLocApocrita/Cluster_GIPM_SSC/**"

for path in glob.glob(path, recursive=True):
    list_all.append(path)
    
#list with only files, not folders
cl_file_list = []

for element in list_all:
    if '.csv' in element:
        cl_file_list.append(element)

cl_dfs = []

for file in cl_file_list:
    df = pd.read_csv(file,encoding='utf-8', names = ['datetime', 'GIPM X (RE)', 'GIPM Y (RE)', 'GIPM Z (RE)'])
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace = True)
    cl_dfs.append(df)

In [24]:
#add cone angle and MA to each row of one cluster df

ma_list = []
cone_a_list = []

#cl_1 = cl_dfs[0]

#ind_list = cl_1.index

#obs = ind_list[2]

#if obs in omni_all.index:
#    print('yes!')
#else:
#    print('no :( ')
    
for i in cl_dfs[0].index:
    if i in omni_all.index:
        ma_temp = omni_all['M_A'].loc[i]
        cone_a_temp = omni_all['cone angle'].loc[i]
        ma_list.append(ma_temp)
        cone_a_list.append(cone_a_temp)
    else:
        ma_list.append(np.nan)
        cone_a_list.append(np.nan)

cl_dfs[0]['MA'] = ma_list
cl_dfs[0]['cone angle'] = cone_a_list

print(cl_dfs[0])

                     GIPM X (RE)  GIPM Y (RE)  GIPM Z (RE)         MA  \
datetime                                                                
2010-02-01 00:00:30    15.000046     9.643389    -7.925858   9.833333   
2010-02-01 00:01:30    15.005020     9.685043    -7.887440   9.871429   
2010-02-01 00:02:30    15.025190     9.693943    -7.897808   9.950000   
2010-02-01 00:03:30    15.052010     9.731482    -7.895245  10.033333   
2010-02-01 00:04:30    15.092709     9.760937    -7.903781  10.130000   
...                          ...          ...          ...        ...   
2010-07-30 21:08:30     1.262483     2.040008    -1.642276  11.855556   
2010-07-30 21:09:30     1.253225     2.100964    -1.658716  11.900000   
2010-07-30 21:10:30     1.243143     2.180931    -1.657347  11.875000   
2010-07-30 21:11:30     1.232838     2.230212    -1.669695  11.814286   
2010-07-30 21:12:30     1.222346     2.279131    -1.681765  11.700000   

                     cone angle  
datetime        

In [33]:
#make new dataframes split by 1. cone angle and 2. MA

df_radial = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] < 30) & (cl_dfs[0]['GIPM X (RE)'].notnull())]
df_spiral = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] > 30) & (cl_dfs[0]['cone angle'] < 60) & (cl_dfs[0]['GIPM X (RE)'].notnull())]
df_perpendicular = cl_dfs[0].loc[(cl_dfs[0]['cone angle'] > 60) & (cl_dfs[0]['GIPM X (RE)'].notnull())]

#now break up by MA. MA ranges <5, 5-8, 8-11, 11-14, 14-18, 18-22, >22

df_rad_less5 = df_radial.loc[(df_radial['MA'] < 5)]
df_rad_5_8 = df_radial.loc[((df_radial['MA'] >= 5) & df_radial['MA'] < 8)]
df_rad_8_11 = df_radial.loc[(df_radial['MA'] >= 8) & (df_radial['MA'] < 11)]
df_rad_11_14 = df_radial.loc[(df_radial['MA'] >= 11) & (df_radial['MA'] < 14)]
df_rad_14_18 = df_radial.loc[(df_radial['MA'] >= 14) & (df_radial['MA'] < 18)]
df_rad_18_22 = df_radial.loc[(df_radial['MA'] >= 18) & (df_radial['MA'] < 22)]
df_rad_22more = df_radial.loc[(df_radial['MA'] >= 22)]

In [34]:
df_rad_8_11

,GIPM X (RE),GIPM Y (RE),GIPM Z (RE),MA,cone angle
datetime,,,,,
2010-02-02 17:12:30,9.588712,-1.303440,-13.422989,8.000000,26.177950
2010-02-02 17:13:30,9.617415,-0.907936,-13.418142,8.000000,22.941555
2010-02-03 05:36:30,14.546596,11.316425,4.178203,8.966667,29.496900
2010-02-03 05:37:30,14.501306,11.100680,4.754120,8.966667,29.496900
2010-02-03 05:38:30,14.556244,11.156160,4.596652,8.900000,28.489946
...,...,...,...,...,...
2010-07-05 23:18:30,0.654786,0.884817,0.226730,10.040000,25.791283
2010-07-05 23:19:30,0.711420,0.841314,0.254218,10.080000,28.250005
2010-07-26 07:40:30,0.570379,-1.057739,0.025028,10.780000,26.507891
